In [2]:
import pandas as pd

In [1]:
from sentence_transformers import SentenceTransformer
SentenceTransformer("intfloat/multilingual-e5-large")

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'XLMRobertaModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [3]:
df = pd.read_json(r"data\parsed_data.json")

In [4]:
df

,Документ,№,ФИО,Время,Текст,Исполнитель,Отработка,Дата,Файл_источник
0,1. СТЕНОГРАММА. GR.,1,Лобов,23:58:00,"Так, давайте. Чернышенко, да, первый доклад? В...",,,17.07.2024,1. СТЕНОГРАММА. GR. 17.07.2024.docx
1,1. СТЕНОГРАММА. GR.,2,Васильев,23:59:00,"Ну мы составили список тех, через кого мы идем...",,,17.07.2024,1. СТЕНОГРАММА. GR. 17.07.2024.docx
2,1. СТЕНОГРАММА. GR.,3,Лобов,00:00,Задача – нам нужна встреча с Чернышенко. Я уже...,,,17.07.2024,1. СТЕНОГРАММА. GR. 17.07.2024.docx
3,1. СТЕНОГРАММА. GR.,4,Другой,01:00,"У меня че-то замучили с этой раскладкой, кажды...",,,17.07.2024,1. СТЕНОГРАММА. GR. 17.07.2024.docx
4,1. СТЕНОГРАММА. GR.,5,Лобов,01:29,Зудин достал. Ты можешь сделать стандартный сп...,Хитров\nЗудин\n(Представить В.Г. Лобову список...,,17.07.2024,1. СТЕНОГРАММА. GR. 17.07.2024.docx
...,...,...,...,...,...,...,...,...,...
72783,стенограмма КПН [1],279,"Еще есть предложение, сейчас тестируем гипотез...",02:14:00,"Еще есть предложение, сейчас тестируем гипотез...",,,11.08.2023,стенограмма КПН 11.08.2023[1].docx
72784,стенограмма КПН [1],280,"Я соглашаюсь, в стратегию – раздел кадровая по...",02:15:00,"Я соглашаюсь, в стратегию – раздел кадровая по...",Корчак \n(в стратегию),,11.08.2023,стенограмма КПН 11.08.2023[1].docx
72785,стенограмма КПН [1],281,Ну тэф тесты провести.,02:16:00,Ну тэф тесты провести.,,,11.08.2023,стенограмма КПН 11.08.2023[1].docx
72786,стенограмма КПН [1],282,"Есть еще для менеджеров, для руководителей 250...",02:17:00,"Есть еще для менеджеров, для руководителей 250...",,,11.08.2023,стенограмма КПН 11.08.2023[1].docx


In [6]:
df['Документ'].nunique()

93

In [5]:
import os
import time
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from openai import APIConnectionError, APITimeoutError, APIError
import httpx

# ===== НАСТРОЙКИ OPENROUTER =====
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
    timeout=300.0,  # Увеличиваем таймаут до 5 минут для длинных запросов
)

df["Дата"] = df["Дата"].astype(str)
output_path = "meeting_summaries.json"

# Загружаем существующие результаты, если файл есть
summaries = []
processed_docs = set()  # Множество успешно обработанных документов

if os.path.exists(output_path):
    try:
        with open(output_path, "r", encoding="utf-8") as f:
            existing_summaries = json.load(f)
            summaries = existing_summaries.copy()
            
            # Собираем список успешно обработанных документов (без ошибок)
            for item in existing_summaries:
                doc_name = str(item.get("Документ", ""))
                summary_text = str(item.get("Сводка", ""))
                
                # Если сводка не содержит [ОШИБКА], значит документ успешно обработан
                if doc_name and "[ОШИБКА]" not in summary_text:
                    processed_docs.add(doc_name)
            
            print(f"📂 Загружено {len(existing_summaries)} записей из {output_path}")
            print(f"✅ Уже успешно обработано документов: {len(processed_docs)}")
            print(f"🔄 Будет обработано заново (с ошибками или новые): {len([s for s in existing_summaries if '[ОШИБКА]' in str(s.get('Сводка', ''))])}")
    except Exception as e:
        print(f"⚠️  Не удалось загрузить существующий файл {output_path}: {e}")
        print("   Начинаем обработку с нуля...")
        summaries = []
        processed_docs = set()
else:
    print(f"📝 Файл {output_path} не найден. Начинаем обработку с нуля...")

# ОГРАНИЧЕНИЕ НА РАЗМЕР ОДНОГО ЗАПРОСА (по символам в records)
MAX_RECORD_CHARS = 128000  # можно уменьшить/увеличить при необходимости

# Настройки retry
MAX_RETRIES = 2  # Максимальное количество попыток
INITIAL_RETRY_DELAY = 2  # Начальная задержка в секундах
MAX_RETRY_DELAY = 60  # Максимальная задержка в секундах


def chunk_records(records, max_chars=MAX_RECORD_CHARS):
    """Делим список реплик на чанки так, чтобы суммарная длина JSON-строк была < max_chars."""
    chunks = []
    current_chunk = []
    current_len = 0

    for r in records:
        s = json.dumps(r, ensure_ascii=False)
        if current_chunk and current_len + len(s) > max_chars:
            chunks.append(current_chunk)
            current_chunk = [r]
            current_len = len(s)
        else:
            current_chunk.append(r)
            current_len += len(s)

    if current_chunk:
        chunks.append(current_chunk)

    return chunks


def make_api_request_with_retry(client, model, messages, temperature=0.4, max_retries=MAX_RETRIES):
    """
    Выполняет запрос к API с автоматическими повторами при ошибках соединения.
    
    Args:
        client: OpenAI клиент
        model: Название модели
        messages: Сообщения для запроса
        temperature: Температура
        max_retries: Максимальное количество попыток
    
    Returns:
        Response от API
    
    Raises:
        Exception: Если все попытки исчерпаны
    """
    last_exception = None
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
            )
            return response
            
        except (APIConnectionError, APITimeoutError, ConnectionError, httpx.ConnectError, httpx.TimeoutException) as e:
            last_exception = e
            error_type = type(e).__name__
            
            # Экспоненциальная задержка с джиттером
            delay = min(INITIAL_RETRY_DELAY * (2 ** attempt), MAX_RETRY_DELAY)
            # Добавляем случайный джиттер (±20%)
            import random
            delay = delay * (0.8 + 0.4 * random.random())
            
            if attempt < max_retries - 1:
                print(f"  ⚠️  Попытка {attempt + 1}/{max_retries} не удалась ({error_type}). Повтор через {delay:.1f} сек...")
                time.sleep(delay)
            else:
                print(f"  ❌ Все {max_retries} попыток исчерпаны. Последняя ошибка: {error_type}")
                
        except APIError as e:
            # Для других API ошибок (400, 500 и т.д.) тоже делаем retry, но с меньшим количеством попыток
            last_exception = e
            error_code = getattr(e, 'status_code', None) or getattr(e, 'code', None)
            
            # Для 500 ошибок делаем retry
            if error_code in [500, 502, 503, 504] and attempt < max_retries - 1:
                delay = min(INITIAL_RETRY_DELAY * (2 ** attempt), MAX_RETRY_DELAY)
                import random
                delay = delay * (0.8 + 0.4 * random.random())
                print(f"  ⚠️  Серверная ошибка {error_code}. Попытка {attempt + 1}/{max_retries}. Повтор через {delay:.1f} сек...")
                time.sleep(delay)
            else:
                # Для других ошибок (400, 429 и т.д.) не делаем retry
                raise e
                
        except Exception as e:
            # Для неизвестных ошибок тоже пробуем повторить
            last_exception = e
            error_type = type(e).__name__
            
            if attempt < max_retries - 1:
                delay = min(INITIAL_RETRY_DELAY * (2 ** attempt), MAX_RETRY_DELAY)
                import random
                delay = delay * (0.8 + 0.4 * random.random())
                print(f"  ⚠️  Неожиданная ошибка ({error_type}): {str(e)[:100]}. Попытка {attempt + 1}/{max_retries}. Повтор через {delay:.1f} сек...")
                time.sleep(delay)
            else:
                raise e
    
    # Если все попытки исчерпаны
    raise last_exception


grouped = df.groupby("Документ")

for doc_name, group in tqdm(grouped, desc="Summarizing meetings"):
    doc_name_str = str(doc_name)
    date = group["Дата"].iloc[0]
    records = group[["№", "ФИО", "Время", "Текст", "Исполнитель"]].to_dict(orient="records")

    # Пропускаем уже успешно обработанные документы
    if doc_name_str in processed_docs:
        continue  # Пропускаем этот документ

    try:
        # 1) Разбиваем реплики на чанки
        record_chunks = chunk_records(records)
        partial_summaries = []

        # 2) Делаем сводку по каждому чанку
        for i, chunk in enumerate(record_chunks, start=1):
            chunk_prompt = f"""
Ты — аналитик, который изучает стенограммы встреч.
Дата встречи: {date}
Документ: {doc_name}
Часть стенограммы: {i} из {len(record_chunks)}.

Ниже таблица реплик (фрагмент, список словарей):

{chunk}

Твоя задача — кратко законспектировать ЭТУ ЧАСТЬ встречи:
- основные темы и тезисы,
- важные поручения,
- ключевые участники и их роли.

Формат вывода:
### Часть {i} — краткое резюме:
- Темы: ...
- Тезисы: ...
- Поручения: ...
- Участники: ...
"""
            response = make_api_request_with_retry(
                client=client,
                model="openai/gpt-4o",
                messages=[{"role": "user", "content": chunk_prompt}],
                temperature=0.4,
            )
            part_summary = response.choices[0].message.content
            partial_summaries.append(part_summary)

            # немного подождём между запросами
            time.sleep(1.5)

        # 3) Финальная сводка по всем частям
        combined_text = "\n\n".join(
            f"Сводка части {i+1}:\n{ps}" for i, ps in enumerate(partial_summaries)
        )

        final_prompt = f"""
Ты — аналитик, который изучает стенограммы встреч.
Дата встречи: {date}
Документ: {doc_name}

Вот набор кратких сводок по разным частям одной и той же встречи:

{combined_text}

На основе этих частичных сводок составь ОДНО целостное структурированное резюме по всей встрече.

Формат вывода:
## Встреча от {date}, Документ {doc_name}

### Ключевые темы:
- ...

### Основные тезисы:
- ...

### Поручения:
- {{ФИО говорящего}} поручил(а) {{ФИО исполнителя}}: {{суть задачи}}

### Участники:
- ...

### Важные упоминания:
- ...

### Итог:
- ...
"""
        final_response = make_api_request_with_retry(
            client=client,
            model="openai/gpt-4o",
            messages=[{"role": "user", "content": final_prompt}],
            temperature=0.4,
        )
        final_summary_text = final_response.choices[0].message.content

        # Обновляем или добавляем запись
        summary_entry = {
            "Документ": doc_name_str,
            "Дата": date,
            "Сводка": final_summary_text,
        }
        
        # Ищем существующую запись для обновления
        found = False
        for i, item in enumerate(summaries):
            if str(item.get("Документ", "")) == doc_name_str:
                summaries[i] = summary_entry
                found = True
                break
        
        if not found:
            summaries.append(summary_entry)

    except Exception as e:
        error_msg = str(e)
        error_type = type(e).__name__
        print(f"❌ Ошибка при обработке документа {doc_name_str}: [{error_type}] {error_msg[:200]}")
        
        error_entry = {
            "Документ": doc_name_str,
            "Дата": date,
            "Сводка": f"[ОШИБКА] {error_type}: {error_msg}",
        }
        
        # Обновляем или добавляем запись об ошибке
        found = False
        for i, item in enumerate(summaries):
            if str(item.get("Документ", "")) == doc_name_str:
                summaries[i] = error_entry
                found = True
                break
        
        if not found:
            summaries.append(error_entry)

    # поэтапная запись после каждого документа
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(summaries, f, ensure_ascii=False, indent=2)

    # Увеличиваем задержку между документами для снижения нагрузки
    time.sleep(2.0)

print(f"Сводки сохраняются по мере обработки в {output_path}")

📂 Загружено 93 записей из meeting_summaries.json
✅ Уже успешно обработано документов: 92
🔄 Будет обработано заново (с ошибками или новые): 1


Summarizing meetings:  41%|████      | 38/93 [00:19<00:00, 85.19it/s]

  ⚠️  Попытка 1/2 не удалась (APIConnectionError). Повтор через 2.1 сек...


Summarizing meetings: 100%|██████████| 93/93 [12:38<00:00,  8.16s/it]

Сводки сохраняются по мере обработки в meeting_summaries.json


In [8]:
result = pd.read_json("meeting_summaries.json")

In [9]:
result

,Документ,Название документа,Дата,Сводка
0,1. СТЕНОГРАММА. GR.,1. СТЕНОГРАММА. GR.,17.07.2024,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
1,1. СТЕНОГРАММА.GR. (1),1. СТЕНОГРАММА.GR. (1),24.07.2024,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
2,1. Стенограмма GR,1. Стенограмма GR,08.05.2024,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
3,1. Стенограмма GR-комитет,1. Стенограмма GR-комитет,03.04.2024,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
4,1. Стенограмма GR. Отработка ПМЭФ,1. Стенограмма GR. Отработка ПМЭФ,12.06.2024,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
...,...,...,...,...
88,Стенограммы КПН,Стенограммы КПН,17.01.2025,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
89,Стенограммы КПН2,Стенограммы КПН2,19.05.2025,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
90,стенограмма GR,стенограмма GR,05.07.2023,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
91,стенограмма КПН,стенограмма КПН,11.08.2023,[ОШИБКА] \n\nYou tried to access openai.ChatCo...
